In [1]:
# import libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
import pickle
from sklearn.metrics import accuracy_score, classification_report, roc_auc_score
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
import joblib

In [68]:
# load the dataset
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/DeepMammoAI_sl_ft.csv')
df

,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,concavity_mean,concave points_mean,radius_worst,perimeter_worst,area_worst,concave points_worst
0,M,17.99,10.38,122.80,1001.0,0.30010,0.14710,25.380,184.60,2019.0,0.2654
1,M,20.57,17.77,132.90,1326.0,0.08690,0.07017,24.990,158.80,1956.0,0.1860
2,M,19.69,21.25,130.00,1203.0,0.19740,0.12790,23.570,152.50,1709.0,0.2430
3,M,11.42,20.38,77.58,386.1,0.24140,0.10520,14.910,98.87,567.7,0.2575
4,M,20.29,14.34,135.10,1297.0,0.19800,0.10430,22.540,152.20,1575.0,0.1625
...,...,...,...,...,...,...,...,...,...,...,...
564,M,21.56,22.39,142.00,1479.0,0.24390,0.13890,25.450,166.10,2027.0,0.2216
565,M,20.13,28.25,131.20,1261.0,0.14400,0.09791,23.690,155.00,1731.0,0.1628
566,M,16.60,28.08,108.30,858.1,0.09251,0.05302,18.980,126.70,1124.0,0.1418
567,M,20.60,29.33,140.10,1265.0,0.35140,0.15200,25.740,184.60,1821.0,0.2650


In [69]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 569 entries, 0 to 568
Data columns (total 11 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   diagnosis             569 non-null    object 
 1   radius_mean           569 non-null    float64
 2   texture_mean          569 non-null    float64
 3   perimeter_mean        569 non-null    float64
 4   area_mean             569 non-null    float64
 5   concavity_mean        569 non-null    float64
 6   concave points_mean   569 non-null    float64
 7   radius_worst          569 non-null    float64
 8   perimeter_worst       569 non-null    float64
 9   area_worst            569 non-null    float64
 10  concave points_worst  569 non-null    float64
dtypes: float64(10), object(1)
memory usage: 49.0+ KB


In [70]:
df.describe()

,radius_mean,texture_mean,perimeter_mean,area_mean,concavity_mean,concave points_mean,radius_worst,perimeter_worst,area_worst,concave points_worst
count,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000
mean,14.127381,19.289649,91.969033,654.889104,0.088799,0.048919,16.269190,107.261213,880.583128,0.114606
std,3.523928,4.301036,24.298981,351.914129,0.079720,0.038803,4.833242,33.602542,569.356993,0.065732
min,6.980000,9.710000,43.790000,143.500000,0.000000,0.000000,7.930000,50.410000,185.200000,0.000000
25%,11.700000,16.170000,75.170000,420.300000,0.029560,0.020310,13.010000,84.110000,515.300000,0.064930
50%,13.370000,18.840000,86.240000,551.100000,0.061540,0.033500,14.970000,97.660000,686.500000,0.099930
75%,15.780000,21.800000,104.100000,782.700000,0.130700,0.074000,18.790000,125.400000,1084.000000,0.161400
max,28.110000,39.280000,188.500000,2501.000000,0.426800,0.201200,36.040000,251.200000,4254.000000,0.291000


In [71]:
df.columns

Index(['diagnosis', 'radius_mean', 'texture_mean', 'perimeter_mean',
       'area_mean', 'concavity_mean', 'concave points_mean', 'radius_worst',
       'perimeter_worst', 'area_worst', 'concave points_worst'],
      dtype='object')

In [96]:
X = df.drop('diagnosis', axis=1)
y = df['diagnosis']

# label encoding
df['diagnosis'] = pd.Categorical(df['diagnosis']).codes
df['diagnosis']



,diagnosis
0,1
1,1
2,1
3,1
4,1
...,...
564,1
565,1
566,1
567,1


In [97]:
X = X.astype(np.float32) # convert features to floats
y = pd.Categorical(y).codes
y = y.astype(np.int32) # convert features to integers

In [98]:
# train, test and split the dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [99]:
# Normalize features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [100]:
# Train model
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)
y_pred = rf_model.predict(X_test)
Accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {Accuracy}")

# # Save model and scaler
# pickle.dump(rf_model, open("deepmammoai_model.pkl", "wb"))
# pickle.dump(scaler, open("scaler.pkl", "wb"))

Accuracy: 0.956140350877193


In [101]:
# import libraries
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier

# train model
svm_model = SVC()
svm_model.fit(X_train, y_train)
y_pred = svm_model.predict(X_test)
Accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {Accuracy}")

knn_model = KNeighborsClassifier()
knn_model.fit(X_train, y_train)
y_pred = knn_model.predict(X_test)
Accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {Accuracy}")

# Save model and scaler
pickle.dump(knn_model, open("deepmammoai_model.pkl", "wb"))
pickle.dump(scaler, open("scaler.pkl", "wb"))

Accuracy: 0.9736842105263158
Accuracy: 0.9736842105263158


In [89]:
# # Predicting the breast cancer status using knn and list of features

# new_data_input = [17.99, 10.38, 122.8, 1001, 0.3001, 0.1471, 25.38, 184.6, 2019, 0.2654]  # Example feature values

# # Reshape the list into a 2D array for prediction
# new_data_input = np.array(new_data_input).reshape(1, -1)


# # Perform inference
# prediction = rf_model.predict(new_data_input)
# print(f"Prediction for the new data input: {prediction}")


Prediction for the new data input: ['M']


----------------------------------------------------------------------------------------

In [103]:
# Define the model
model = Sequential([
    Dense(64, activation="relu", input_shape=(X_train.shape[1],)),
    Dropout(0.3),
    Dense(32, activation="relu"),
    Dropout(0.3),
    Dense(1, activation="sigmoid")
])

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy',
                       tf.keras.metrics.Precision(name='precision'),
                       tf.keras.metrics.Recall(name='recall'),
                       tf.keras.metrics.AUC(name='auc')])

# Early stopping
early_stop = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Train model
history = model.fit(X_train, y_train,
                    validation_split=0.2,
                    epochs=50,
                    batch_size=32,
                    callbacks=[early_stop],
                    verbose=1)

# Evaluate the model
y_pred = (model.predict(X_test) > 0.5).astype(int)
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))
print("AUC-ROC:", roc_auc_score(y_test, y_pred))

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 11s 185ms/step - accuracy: 0.6557 - auc: 0.5160 - loss: 0.6820 - precision: 0.5791 - recall: 0.2429 - val_accuracy: 0.9231 - val_auc: 0.9864 - val_loss: 0.5648 - val_precision: 1.0000 - val_recall: 0.7879
Epoch 2/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 1s 44ms/step - accuracy: 0.8247 - auc: 0.9152 - loss: 0.5488 - precision: 0.8329 - recall: 0.6360 - val_accuracy: 0.9451 - val_auc: 0.9875 - val_loss: 0.4420 - val_precision: 0.9375 - val_recall: 0.9091
Epoch 3/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 1s 35ms/step - accuracy: 0.9252 - auc: 0.9816 - loss: 0.4289 - precision: 0.9579 - recall: 0.8507 - val_accuracy: 0.9451 - val_auc: 0.9851 - val_loss: 0.3311 - val_precision: 0.9118 - val_recall: 0.9394
Epoch 4/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 1s 41ms/step - accuracy: 0.9405 - auc: 0.9838 - loss: 0.3146 - precision: 0.9540 - recall: 0.8890 - val_accuracy: 0.9451 - val_auc: 0.9862 - val_loss: 0.2511 - val_precision: 0.9118 - val_recall: 0.9394
Epoch 5/50
12/12 ━━━━━━━━━━━━━━━━━

In [104]:
# Save the trained model
model.save("deepmammoai_model.h5")

In [105]:
!pip install flask streamlit flask_ngrok

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.8/9.8 MB 51.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 58.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 5.8 MB/s eta 0:00:00


In [28]:
# !wget -q -O - ipv4.icanhazip.com

34.75.74.101


In [106]:
!curl https://loca.lt/mytunnelpassword

34.168.93.28

In [ ]:
!streamlit run /content/app.py & npx localtunnel --port 8501



⠙⠹⠸⠼
  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.168.93.28:8501

⠴⠦⠧your url is: https://eighty-aliens-jog.loca.lt
2025-04-04 09:25:13.040 Uncaught exception GET /_stcore/stream (127.0.0.1)
HTTPServerRequest(protocol='http', host='eighty-aliens-jog.loca.lt', method='GET', uri='/_stcore/stream', version='HTTP/1.1', remote_ip='127.0.0.1')
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/streamlit/web/bootstrap.py", line 347, in run
    if asyncio.get_running_loop().is_running():
       ^^^^^^^^^^^^^^^^^^^^^^^^^^
RuntimeError: no running event loop

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/tornado/websocket.py", line 938, in _accept_connection
    open_result = handler.open(*handler.open_args, **handler.open_kwargs)
                  

In [26]:
# from google.colab.output import eval_js
# print(eval_js("google.colab.kernel.proxyPort(5000)"))

In [27]:
# # Create a Flask app
# from flask import Flask, render_template, request
# from flask_ngrok import run_with_ngrok

# app = Flask(__name__, template_folder='/content/Templates')

# @app.route("/")
# def home():
#     return render_template("index.html")

# run_with_ngrok(app)
# app.run()